In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.cluster import SpectralClustering
from sklearn.metrics import silhouette_score
from scipy.stats import gaussian_kde
import ipywidgets as widgets
from IPython.display import display
import warnings
from sklearn.exceptions import ConvergenceWarning
from statsmodels.tsa.seasonal import seasonal_decompose

# --------------------------
# DATA GENERATION FUNCTIONS
# --------------------------

def generate_blobs(n_samples=300):
    """Generate clustered blob data."""
    return datasets.make_blobs(n_samples=n_samples, centers=3, cluster_std=1.0, n_features=3, random_state=42)[0]

def generate_moons(n_samples=300):
    """Generate moon-shaped data."""
    data, _ = datasets.make_moons(n_samples=n_samples, noise=0.05)
    return np.c_[data, np.zeros(n_samples)]  # Add a dummy zero third dimension

def generate_swiss_roll(n_samples=300):
    """Generate swiss roll data."""
    data, _ = datasets.make_swiss_roll(n_samples=n_samples, noise=0.05)
    return data

def generate_s_curve(n_samples=300):
    """Generate S-curve data."""
    data, _ = datasets.make_s_curve(n_samples=n_samples, noise=0.05)
    return data

def generate_random(n_samples=300):
    """Generate random data."""
    return np.random.rand(n_samples, 3)

def generate_wine():
    """Generate wine data and project it to 3 dimensions using PCA."""
    wine_data = datasets.load_wine().data
    pca = PCA(n_components=3)
    transformed_wine_data = pca.fit_transform(wine_data)
    return transformed_wine_data

def generate_high_dim_small_sample():
    """Generate high dimensional small sample data."""
    np.random.seed(42)  # for reproducibility
    return np.random.randn(50, 100)

def generate_time_series(n_samples=300):
    """Generate synthetic time series data."""
    t = np.linspace(0, 4 * np.pi, n_samples)
    return np.sin(t) + 0.5 * np.random.randn(n_samples)


# Available datasets
DATA_OPTIONS = {
    "Blobs": generate_blobs,
    "Moons": generate_moons,
    "Swiss Roll": generate_swiss_roll,
    "S-Curve": generate_s_curve,
    "Random": generate_random,
    "Wine": generate_wine,
    "High Dimensional Small Sample": generate_high_dim_small_sample,
    "Time Series": generate_time_series
}

# --------------------------
# UTILITY FUNCTIONS
# --------------------------

def gaussian_similarity_matrix(data, sigma):
    """Compute the Gaussian similarity matrix."""
    pairwise_dists = np.sum(data**2, 1).reshape(-1, 1) + np.sum(data**2, 1) - 2 * data @ data.T
    return np.exp(-pairwise_dists / (2 * sigma**2))

def compute_laplacian(similarity_matrix):
    """Compute the Laplacian matrix."""
    D = np.diag(np.sum(similarity_matrix, axis=1))
    return D - similarity_matrix

def generate_vector_field(laplacian, data):
    """Generate the vector field using Fiedler vector (2nd smallest eigenvector of Laplacian)."""
    eigvals, eigvecs = np.linalg.eigh(laplacian)
    fiedler_vector = eigvecs[:, 1]
    X, Y, Z = data[:, 0], data[:, 1], data[:, 2]
    U, V, W = fiedler_vector, fiedler_vector, fiedler_vector
    return X, Y, Z, U, V, W

def decompose_time_series(time_series, period=12):
    """
    Decomposes a time series into its trend, seasonal, and residual components.
    Args:
    - time_series (array-like): The time series data.
    - period (int): The frequency for the seasonal decomposition.
    
    Returns:
    - result (DecomposeResult): The decomposition result containing trend, seasonal, and residual components.
    """
    result = seasonal_decompose(time_series, model='additive', period=period)
    return result

# --------------------------
# VISUALIZATION FUNCTION
# --------------------------

def visualize(data_type, sigma, n_clusters, bin_size):
    """Main visualization function."""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # Suppress warnings
        
        # Generate data
        data = DATA_OPTIONS[data_type]()

        if data_type == "Time Series":
            # Decompose the time series
            decomposition = decompose_time_series(data)

            # Plot the original time series, trend, seasonal, and residual components
            fig, axes = plt.subplots(4, 1, figsize=(10, 8))

            axes[0].plot(data, label="Original")
            axes[0].legend(loc="best")
            axes[0].set_title("Original Time Series")

            axes[1].plot(decomposition.trend, label="Trend")
            axes[1].legend(loc="best")
            axes[1].set_title("Trend Component")

            axes[2].plot(decomposition.seasonal, label="Seasonal")
            axes[2].legend(loc="best")
            axes[2].set_title("Seasonal Component")

            axes[3].plot(decomposition.resid, label="Residual")
            axes[3].legend(loc="best")
            axes[3].set_title("Residual Component")

            plt.tight_layout()
            plt.show()
            return  # Exit the function to avoid other visualizations for time series data

        # Initialize a figure for plotting
        fig = plt.figure(figsize=(16, 20))

        # --------------------------
        # 1. PCA Visualization
        # --------------------------
        
        ax1 = fig.add_subplot(3, 2, 1, projection='3d')
        n_samples, n_features = data.shape

        # Use Gram matrix if number of samples is less than number of features
        if n_samples < n_features:
            G = np.dot(data, data.T)
            eigvals, eigvecs = np.linalg.eigh(G)
            pca_data = np.dot(eigvecs.T, data)
        else:
            pca_model = PCA(n_components=3)
            pca_data = pca_model.fit_transform(data)

        sc1 = ax1.scatter(pca_data[:, 0], pca_data[:, 1], pca_data[:, 2], c=pca_data[:, 2], cmap='rainbow', marker='o', s=100, alpha=0.6)

        # Plot eigenvectors if we used the standard PCA approach
        if n_samples >= n_features:
            for length, vector in zip(pca_model.explained_variance_, pca_model.components_):
                v = vector * 3 * np.sqrt(length)
                ax1.quiver(0, 0, 0, v[0], v[1], v[2], arrow_length_ratio=0.1, color='red')
            ax1.text2D(0.05, 0.95, f"Explained Variance Ratios: {pca_model.explained_variance_ratio_}", transform=ax1.transAxes)
        
        ax1.set_title('PCA Visualization with Eigenvectors')
        fig.colorbar(sc1, ax=ax1, label='PCA Component 3 Value')

        # --------------------------
        # 2. Spectral Clustering Visualization
        # --------------------------
        
        ax2 = fig.add_subplot(3, 2, 2, projection='3d')

        # Compute similarity matrix and Laplacian
        similarity_matrix = gaussian_similarity_matrix(data, sigma)
        laplacian_matrix = compute_laplacian(similarity_matrix)

        # Perform spectral clustering
        clustering_labels = SpectralClustering(n_clusters=n_clusters, affinity='precomputed').fit_predict(similarity_matrix)

        sc2 = ax2.scatter(data[:, 0], data[:, 1], data[:, 2], c=clustering_labels, cmap='rainbow', marker='o', s=100, alpha=0.6)

        # Compute silhouette score
        if n_samples < n_features:
            silhouette_avg = silhouette_score(pca_data[:, :3], clustering_labels)  # Use PCA-reduced data for silhouette
        else:
            silhouette_avg = silhouette_score(data, clustering_labels)
        
        ax2.text2D(0.05, 0.95, f"Silhouette Score: {silhouette_avg:.2f}", transform=ax2.transAxes)
        ax2.set_title('Spectral Clustering Visualization')
        fig.colorbar(sc2, ax=ax2, label='Cluster ID')

        # --------------------------
        # 3. Original Data with Vector Field
        # --------------------------
        
        ax3 = fig.add_subplot(3, 2, 3, projection='3d')
        
        # Generate vector field using Fiedler vector
        X, Y, Z, U, V, W = generate_vector_field(laplacian_matrix, data)
        
        sc3 = ax3.scatter(data[:, 0], data[:, 1], data[:, 2], c=data[:, 2], cmap='rainbow', marker='o', s=100, alpha=0.6)
        ax3.quiver(X, Y, Z, U, V, W, length=0.1, normalize=True, color='k', linewidth=0.5, alpha=0.5)
        ax3.set_title('Original Data with Vector Field and Eigenvectors')
        fig.colorbar(sc3, ax=ax3, label='Z-axis Value')

        # Display basic statistics
        metrics = [
            f"X-axis: Min: {data[:,0].min():.2f}, Max: {data[:,0].max():.2f}, Mean: {data[:,0].mean():.2f}",
            f"Y-axis: Min: {data[:,1].min():.2f}, Max: {data[:,1].max():.2f}, Mean: {data[:,1].mean():.2f}",
            f"Z-axis: Min: {data[:,2].min():.2f}, Max: {data[:,2].max():.2f}, Mean: {data[:,2].mean():.2f}"
        ]
        for i, metric in enumerate(metrics):
            ax3.text2D(0.05, 0.95 - i * 0.05, metric, transform=ax3.transAxes)

        # --------------------------
        # 4. KDE Visualization
        # --------------------------
        
        ax4 = fig.add_subplot(3, 2, 4, projection='3d')
        
        # Compute and visualize KDE if applicable
        variance_threshold = 1e-4
        is_variance_high_enough = np.all(np.var(data, axis=0) > variance_threshold)
        is_more_samples_than_features = data.shape[0] >= data.shape[1]

        if is_variance_high_enough and is_more_samples_than_features:
            kde_estimator = gaussian_kde(data.T)
            density_values = kde_estimator(data.T)
            sc4 = ax4.scatter(data[:, 0], data[:, 1], data[:, 2], c=density_values, cmap='Reds', s=100)
            fig.colorbar(sc4, ax=ax4, label='Density')
        else:
            ax4.text2D(0.5, 0.5, "KDE not applicable for this dataset", transform=ax4.transAxes, ha='center')
        
        ax4.set_title('KDE Visualization')

        # --------------------------
        # 5. Histogram Visualization
        # --------------------------
        
        ax5 = fig.add_subplot(3, 2, 5, projection='3d')
        
        # Compute and visualize histogram based on data dimensionality
        if data.shape[1] == 3:
            hist_data, edges_data = np.histogramdd(data, bins=(bin_size, bin_size, bin_size))
        elif data.shape[1] == 2:
            hist_data, edges_data = np.histogram2d(data[:, 0], data[:, 1], bins=(bin_size, bin_size))
            hist_data = np.expand_dims(hist_data, axis=2)  # Add a dummy third dimension for the histogram
            edges_data = list(edges_data) + [np.array([0, 1])]
        else:
            hist_data, edges_data = np.histogram(data, bins=bin_size)
            hist_data = np.expand_dims(hist_data, axis=(1,2))  # Add dummy dimensions for 3D visualization
            edges_data = [edges_data, np.array([0, 1]), np.array([0, 1])]
        
        x_edges, y_edges, z_edges = edges_data
        x_positions, y_positions, z_positions = np.meshgrid(x_edges[:-1], y_edges[:-1], z_edges[:-1], indexing="ij")
        x_positions = x_positions.flatten()
        y_positions = y_positions.flatten()
        z_positions = z_positions.flatten()
        dx = dy = dz = bin_size * np.ones_like(z_positions)
        ax5.bar3d(x_positions, y_positions, z_positions, dx, dy, hist_data.flatten(), shade=True, color='cyan', zsort='average')
        ax5.set_title('Histogram Visualization')

        plt.tight_layout()
        plt.show()

# --------------------------
# INTERACTIVE WIDGET
# --------------------------

widgets.interactive(
    visualize,
    data_type=widgets.Dropdown(options=DATA_OPTIONS.keys(), value='Blobs', description='Data Type:'),
    sigma=widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Sigma:'),
    n_clusters=widgets.IntSlider(value=3, min=2, max=10, description='Clusters:'),
    bin_size=widgets.IntSlider(value=10, min=1, max=50, description='Bin Size:')
)


interactive(children=(Dropdown(description='Data Type:', options=('Blobs', 'Moons', 'Swiss Roll', 'S-Curve', '…